# Library


In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from datetime import datetime
import pyexasol

In [2]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# server

In [3]:
# # insider db
# wb = pyexasol.connect(dsn='172.29.135.35/F99FAB2444F86051A9A467F6313FAAB48AF7C4760663430958E3B89A9DC53361:8563', user='sys', password='exasol', compression=True, schema='VSYSD')
# exasol db
ws = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYS')
wb = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSD')
we = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='vsyse')
wbt = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='wbt')

# fold path

In [4]:
drive = 'D'
ods_fold = f'{drive}:/data/big2/ODS'
big2_fold = f'{drive}:/data/big2'
an3_fold = f'{drive}:/data/big2/BD3/an'
raw3_fold = f'{drive}:/data/big2/BD3/raw'

# Load
- 등록&제원 -> result 5등급

## 등록정보(STD_CEG_CAR_MIG) 5등급만

In [5]:
# 8.6s
car = wb.export_to_pandas("SELECT VIN, BSPL_STDG_CD, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD, VHCL_FBCTN_YMD, VHCL_MNG_NO FROM STD_CEG_CAR_MIG WHERE EXHST_GAS_GRD_CD = 'A0505' OR EXHST_GAS_GRD_CD = 'A05T5';")

C:\Users\kbjung\AppData\Roaming\Python\Python39\site-packages\pyexasol\callback.py:42: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  return pandas.read_csv(pipe, skip_blank_lines=False, **kwargs)


In [6]:
car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2378646 entries, 0 to 2378645
Data columns (total 10 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   VIN             object
 1   BSPL_STDG_CD    int64 
 2   VHCL_ERSR_YN    object
 3   MANP_MNG_NO     object
 4   YRIDNW          int64 
 5   VHCTY_CD        object
 6   PURPS_CD2       object
 7   FRST_REG_YMD    int64 
 8   VHCL_FBCTN_YMD  object
 9   VHCL_MNG_NO     object
dtypes: int64(3), object(7)
memory usage: 181.5+ MB


In [7]:
car_ch_col = {
    'VIN':'차대번호', 
    'BSPL_STDG_CD':'법정동코드', 
    'VHCL_ERSR_YN':'차량말소YN',
    'MANP_MNG_NO':'제원관리번호',
    'YRIDNW':'차량연식', 
    'VHCTY_CD':'차종', 
    'PURPS_CD2':'용도', 
    'FRST_REG_YMD':'최초등록일자',
    'VHCL_FBCTN_YMD':'제작일자',
    'VHCL_MNG_NO':'차량관리번호'
}

In [8]:
carr = car.rename(columns=car_ch_col)

In [9]:
carr.columns

Index(['차대번호', '법정동코드', '차량말소YN', '제원관리번호', '차량연식', '차종', '용도', '최초등록일자',
       '제작일자', '차량관리번호'],
      dtype='object')

## 제원정보(STD_CEG_CAR_SRC_MIG)

In [10]:
# 3.8s
src = wb.export_to_pandas("SELECT MANP_MNG_NO, FUEL_CD, VHCTY_TY_CD2 FROM STD_CEG_CAR_SRC_MIG;")

In [11]:
src_ch_col = {
    'MANP_MNG_NO':'제원관리번호', 
    'FUEL_CD':'연료',
    'VHCTY_TY_CD2':'차종유형', 
}

In [12]:
srcr = src.rename(columns=src_ch_col)
srcr.columns

Index(['제원관리번호', '연료', '차종유형'], dtype='object')

## 저감장치 부착이력(STD_DLM_TB_ERP_ATT_HIS)

In [13]:
# 3.0s
att = wb.export_to_pandas("SELECT VIN, RDCDVC_SE_CD FROM STD_DLM_TB_ERP_ATT_HIS;")
att.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1010946 entries, 0 to 1010945
Data columns (total 2 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   VIN           1010946 non-null  object
 1   RDCDVC_SE_CD  1010946 non-null  object
dtypes: object(2)
memory usage: 15.4+ MB


In [14]:
att_ch_col = {
    'VIN':'차대번호', 
    'RDCDVC_SE_CD':'저감장치구분', 
}

In [15]:
attr = att.rename(columns=att_ch_col)
attr.columns

Index(['차대번호', '저감장치구분'], dtype='object')

## 법정동코드(STD_BJCD_INFO)

In [16]:
# 1.3s
code = wb.export_to_pandas("SELECT STDG_CD, STDG_CTPV_CD, STDG_SGG_CD, STDG_CTPV_NM, STDG_SGG_NM FROM STD_BJCD_INFO;")
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46333 entries, 0 to 46332
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   STDG_CD       46333 non-null  int64 
 1   STDG_CTPV_CD  46333 non-null  int64 
 2   STDG_SGG_CD   46333 non-null  int64 
 3   STDG_CTPV_NM  46333 non-null  object
 4   STDG_SGG_NM   46310 non-null  object
dtypes: int64(3), object(2)
memory usage: 1.8+ MB


In [17]:
code.columns

Index(['STDG_CD', 'STDG_CTPV_CD', 'STDG_SGG_CD', 'STDG_CTPV_NM',
       'STDG_SGG_NM'],
      dtype='object')

In [18]:
code_ch_col = {
    'STDG_CD':'법정동코드', 
    'STDG_CTPV_NM':'시도', 
    'STDG_SGG_NM':'시군구', 
    'STDG_CTPV_CD':'시도코드', 
    'STDG_SGG_CD':'시군구코드',
}

In [19]:
coder = code.rename(columns=code_ch_col)
coder.columns

Index(['법정동코드', '시도코드', '시군구코드', '시도', '시군구'], dtype='object')

## 노후차 조기폐차 관리정보(수도권)(STD_DLM_TB_ERP_EARLY_ERASE_AEA)

In [20]:
# 2.4s
aea = wb.export_to_pandas("SELECT VIN, ELPDSRC_STTS_CD, ELPDSRC_LST_APRV_YN, ERSR_YMD FROM STD_DLM_TB_ERP_EARLY_ERASE_AEA;")
aea.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804082 entries, 0 to 804081
Data columns (total 4 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   VIN                  804082 non-null  object 
 1   ELPDSRC_STTS_CD      804082 non-null  object 
 2   ELPDSRC_LST_APRV_YN  683874 non-null  object 
 3   ERSR_YMD             689973 non-null  float64
dtypes: float64(1), object(3)
memory usage: 24.5+ MB


In [21]:
aea.columns

Index(['VIN', 'ELPDSRC_STTS_CD', 'ELPDSRC_LST_APRV_YN', 'ERSR_YMD'], dtype='object')

In [22]:
aea_ch_col = {
    'VIN':'차대번호', 
    'ELPDSRC_STTS_CD':'조기폐차상태코드', 
    'ELPDSRC_LST_APRV_YN':'조기폐차최종승인YN', 
    'ERSR_YMD':'말소일자',
}

In [23]:
aear = aea.rename(columns=aea_ch_col)
aear.columns

Index(['차대번호', '조기폐차상태코드', '조기폐차최종승인YN', '말소일자'], dtype='object')

In [24]:
aear.head()

,차대번호,조기폐차상태코드,조기폐차최종승인YN,말소일자
0,KMCXLS7BP2U626435,A32Y,Y,20120524.00
1,KNAJA5265VA794691,A32Y,Y,20120524.00
2,KPBLC3D81VP018295,A32Y,Y,20120524.00
3,KPDHADMD1YP225493,A32Y,Y,20120524.00
4,KNAMA75122S249083,A32Y,Y,20120529.00


## 노후차 조기폐차 관리정보(수도권외)(STD_DLM_TB_ERP_EARLY_ERASE_LGV)

In [144]:
# 1.8s
lgv = wb.export_to_pandas("SELECT VIN, ELPDSRC_STTS_CD, ELPDSRC_LST_APRV_YN, ERSR_YMD FROM STD_DLM_TB_ERP_EARLY_ERASE_LGV;")
lgv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373371 entries, 0 to 373370
Data columns (total 4 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   VIN                  373371 non-null  object 
 1   ELPDSRC_STTS_CD      373371 non-null  object 
 2   ELPDSRC_LST_APRV_YN  160690 non-null  object 
 3   ERSR_YMD             175443 non-null  float64
dtypes: float64(1), object(3)
memory usage: 11.4+ MB


In [145]:
lgv.columns

Index(['VIN', 'ELPDSRC_STTS_CD', 'ELPDSRC_LST_APRV_YN', 'ERSR_YMD'], dtype='object')

In [146]:
lgv_ch_col = {
    'VIN':'차대번호', 
    'ELPDSRC_STTS_CD':'조기폐차상태코드', 
    'ELPDSRC_LST_APRV_YN':'조기폐차최종승인YN', 
    'ERSR_YMD':'말소일자', 
}

In [147]:
lgvr = lgv.rename(columns=lgv_ch_col)
lgvr.columns

Index(['차대번호', '조기폐차상태코드', '조기폐차최종승인YN', '말소일자'], dtype='object')

In [148]:
lgvr.head()

,차대번호,조기폐차상태코드,조기폐차최종승인YN,말소일자
0,KMHSD81VP4U791871,A32M,NaN,NaN
1,KPBFA2AF16P223800,A32M,NaN,NaN
2,KMFWVH7HP7U772891,A32M,NaN,NaN
3,KMFXKN7BPYU361960,A32M,NaN,NaN
4,KMFXKS7BP2U563724,A32M,NaN,NaN


## 비상시 및 계절제 단속발령(N_IS_ISSUE_DISCLOSURE)

In [30]:
# # 1.8s
# isdis = we.export_to_pandas("SELECT GNFD_NO, VIN, REG_SIDO_CD, REG_SIGNGU_CD FROM ODS_N_IS_ISSUE_DISCLOSURE;")
# isdis.info()

In [183]:
# about 58.4s
name = 'ODS_N_IS_ISSUE_DISCLOSURE(col_sliced)'
file_name = f'{name}.csv'
isdis = pd.read_csv(os.path.join(ods_fold, file_name), usecols=['REGLT_NO', 'GNFD_NO', 'VIN', 'REG_SIDO_CD', 'REG_SIGNGU_CD', 'REGLT_AREA_CD']) # usecols=['REGLT_NO', 'GNFD_NO, 'VIN', 'REG_SIDO_CD', 'REG_SIGNGU_CD', 'REGLT_AREA_CD']
isdis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81512621 entries, 0 to 81512620
Data columns (total 6 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   REGLT_NO       int64 
 1   GNFD_NO        int64 
 2   REG_SIDO_CD    int64 
 3   REG_SIGNGU_CD  int64 
 4   REGLT_AREA_CD  int64 
 5   VIN            object
dtypes: int64(5), object(1)
memory usage: 3.6+ GB


In [184]:
isdis.head()

,REGLT_NO,GNFD_NO,REG_SIDO_CD,REG_SIGNGU_CD,REGLT_AREA_CD,VIN
0,54793831,1838,41,190,41190,KMFXKS7BP3U657943
1,54793839,1838,41,190,41190,KNAMA75432S388470
2,54793843,1838,28,237,41190,KNGDNM9N14K178854
3,54795717,1838,41,130,41190,KMFGA17PP7C072098
4,54793837,1838,41,190,41190,KPBSH3AB18P082348


In [185]:
isdis.columns

Index(['REGLT_NO', 'GNFD_NO', 'REG_SIDO_CD', 'REG_SIGNGU_CD', 'REGLT_AREA_CD',
       'VIN'],
      dtype='object')

In [186]:
isdis_ch_col = {
    'REGLT_NO':'적발번호', 
    'GNFD_NO':'발령번호', 
    'REG_SIDO_CD':'등록시도코드', 
    'REG_SIGNGU_CD':'등록시군구코드', 
    'VIN':'차대번호',
    'REGLT_AREA_CD':'적발지역코드', 
}

In [187]:
isdisr = isdis.rename(columns=isdis_ch_col)
isdisr.columns

Index(['적발번호', '발령번호', '등록시도코드', '등록시군구코드', '적발지역코드', '차대번호'], dtype='object')

## 운행제한 발령정보(N_IS_ISSUE)

In [36]:
# # 1.8s
# isis = we.export_to_pandas("SELECT GNFD_NO, TY_STDR_ID, DNSTY_STDR_ID FROM ODS_N_IS_ISSUE;")
# isis.info()

In [37]:
name = 'ODS_N_IS_ISSUE'
file_name = f'{name}.csv'
isis = pd.read_csv(os.path.join(ods_fold, file_name), usecols=['GNFD_NO', 'TY_STDR_ID', 'DNSTY_STDR_ID']) # usecols=['GNFD_NO', 'TY_STDR_ID', 'DNSTY_STDR_ID']
isis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1785 entries, 0 to 1784
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   GNFD_NO        1785 non-null   int64 
 1   DNSTY_STDR_ID  1785 non-null   object
 2   TY_STDR_ID     1785 non-null   object
dtypes: int64(1), object(2)
memory usage: 42.0+ KB


In [38]:
isis.columns

Index(['GNFD_NO', 'DNSTY_STDR_ID', 'TY_STDR_ID'], dtype='object')

In [39]:
isis_ch_col = {
    'GNFD_NO':'발령번호', 
    'DNSTY_STDR_ID':'농도기준아이디', 
    'TY_STDR_ID':'유형기준아이디', 
}

In [40]:
isisr = isis.rename(columns=isis_ch_col)
isisr.columns

Index(['발령번호', '농도기준아이디', '유형기준아이디'], dtype='object')

In [41]:
# DNSTY_STDR_ID(농도기준아이디) : 실발령(C011), 모의발령(C012)
# TY_STDR_ID(유형기준아이디) : 비상시(T001), 계절제(T002)

## N_IS_PENALTY

In [188]:
# 9.5s
# ispe = ws.export_to_pandas("SELECT REGLT_NO, REGLT_DE FROM N_IS_PENALTY WHERE ROWNUM <= 10000;")
ispe = ws.export_to_pandas("SELECT REGLT_NO, REGLT_DE FROM N_IS_PENALTY;")
ispe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10790105 entries, 0 to 10790104
Data columns (total 2 columns):
 #   Column    Dtype
---  ------    -----
 0   REGLT_NO  int64
 1   REGLT_DE  int64
dtypes: int64(2)
memory usage: 164.6 MB


In [189]:
# # about 2.1s
# name = 'N_IS_PENALTY'
# file_name = f'{name}.csv'
# ispe = pd.read_csv(os.path.join(ods_fold, file_name), usecols=['REGLT_NO, 'REGLT_DE'], nrows=100**2, sep='|') # usecols=['REGLT_NO, 'REGLT_DE']
# ispe.info()

In [190]:
ispe.head()

,REGLT_NO,REGLT_DE
0,40995,20191210
1,40828,20191210
2,40676,20191210
3,40683,20191210
4,42850,20191210


In [191]:
ispe_ch_col = {
    'REGLT_NO':'적발번호', 
    'REGLT_DE':'단속일', 
}

In [192]:
isper = ispe.rename(columns=ispe_ch_col)
isper.columns

Index(['적발번호', '단속일'], dtype='object')

## 운행제한 단속정보(N_US_DISCLOSURE)

In [49]:
# # 1.8s
# ispe = we.export_to_pandas("SELECT GNFD_NO, REGLT_DE FROM N_US_DISCLOSURE WHERE ROWNUM <= 10000;")
# # ispe = we.export_to_pandas("SELECT GNFD_NO, REGLT_DE FROM N_US_DISCLOSURE;")
# ispe.info()

In [282]:
# about 13.5s
name = 'ODS_N_US_DISCLOSURE'
file_name = f'{name}.csv'
usdis = pd.read_csv(os.path.join(ods_fold, file_name), usecols=['NO', 'VIN', 'DISCL_TY', 'REGLT_AREA_CD', 'REG_SIDO_CD', 'REG_SIGNGU_CD']) # usecols=['NO', 'VIN, 'DISCL_TY', 'REGLT_AREA_CD', 'REG_SIDO_CD', 'REG_SIGNGU_CD']
usdis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3090035 entries, 0 to 3090034
Data columns (total 6 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   NO             int64 
 1   REG_SIDO_CD    int64 
 2   REG_SIGNGU_CD  int64 
 3   DISCL_TY       object
 4   REGLT_AREA_CD  int64 
 5   VIN            object
dtypes: int64(4), object(2)
memory usage: 141.5+ MB


In [283]:
usdis.head()

,NO,REG_SIDO_CD,REG_SIGNGU_CD,DISCL_TY,REGLT_AREA_CD,VIN
0,117112,41,110,S,41430,KPBFA2AD12P021974
1,117111,11,0,S,41430,KMHNN81XP2U038507
2,117107,41,410,S,41430,KMFZCD7JP5U094049
3,132833,11,0,S,41130,KMFWVH7HP4U579325
4,6933349,11,0,S,41130,KNAMA75433S411400


In [284]:
usdis.columns

Index(['NO', 'REG_SIDO_CD', 'REG_SIGNGU_CD', 'DISCL_TY', 'REGLT_AREA_CD',
       'VIN'],
      dtype='object')

In [285]:
usdis_ch_dict = {
    'NO':'번호', 
    'VIN':'차대번호', 
    'REG_SIDO_CD':'등록시도코드', 
    'REG_SIGNGU_CD':'등록시군구코드', 
    'DISCL_TY':'적발유형', 
    'REGLT_AREA_CD':'단속지역코드', 
}

In [286]:
usdisr = usdis.rename(columns=usdis_ch_dict)
usdisr.columns

Index(['번호', '등록시도코드', '등록시군구코드', '적발유형', '단속지역코드', '차대번호'], dtype='object')

In [55]:
# DISCL_TY(적발유형) : S(일반 -> 상시), H(고농도)

## N_US_PENALTY

In [344]:
# 1.8s
uspe = ws.export_to_pandas('SELECT "NO", REGLT_CNT, REGLT_YM FROM N_US_PENALTY;')
uspe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 443309 entries, 0 to 443308
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   NO         443309 non-null  int64
 1   REGLT_CNT  443309 non-null  int64
 2   REGLT_YM   443309 non-null  int64
dtypes: int64(3)
memory usage: 10.1 MB


In [345]:
# # about 2.1s
# name = 'N_US_PENALTY'
# file_name = f'{name}.csv'
# uspe = pd.read_csv(os.path.join(ods_fold, file_name), nrows=100**2, sep='|' ) # usecols=['NO', 'REGLT_CNT']
# uspe.info()

In [346]:
uspe['REGLT_CNT'].describe()

count   443309.00
mean         4.54
std          5.08
min          1.00
25%          1.00
50%          2.00
75%          6.00
max         31.00
Name: REGLT_CNT, dtype: float64

In [347]:
uspe.columns

Index(['NO', 'REGLT_CNT', 'REGLT_YM'], dtype='object')

In [348]:
uspe_ch_dict = {
    'NO':'번호', 
    'REGLT_CNT':'적발건수', 
    'REGLT_YM':'적발년월'
}

In [349]:
usper = uspe.rename(columns=uspe_ch_dict)
usper.columns

Index(['번호', '적발건수', '적발년월'], dtype='object')

## 등록이력(CEG_CAR_HISTORY_MIG)

In [62]:
# 1m 10.1s
his = ws.export_to_pandas("SELECT VHCL_ERSR_YN, CHNG_DE, VHMNO FROM CEG_CAR_HISTORY_MIG;")
his.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50044247 entries, 0 to 50044246
Data columns (total 3 columns):
 #   Column        Dtype  
---  ------        -----  
 0   VHCL_ERSR_YN  object 
 1   CHNG_DE       float64
 2   VHMNO         object 
dtypes: float64(1), object(2)
memory usage: 1.1+ GB


In [63]:
his_ch_col = {
    'VHCL_ERSR_YN':'차량말소YN', 
    'CHNG_DE':'변경일자',
    'VHMNO':'차량관리번호'
}

In [64]:
hisr = his.rename(columns=his_ch_col)
hisr.columns

Index(['차량말소YN', '변경일자', '차량관리번호'], dtype='object')

# 전처리

## 중복 차대번호 제거

In [65]:
carr.shape, len(carr['차대번호'].unique())

((2378646, 10), 2378598)

In [66]:
carr = carr.sort_values('최초등록일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
carr.shape

(2378598, 10)

## 차종 코드 변환

In [67]:
cd_dict = {
    'A31M':'이륜', 
    'A31P':'승용', 
    'A31S':'특수', 
    'A31T':'화물', 
    'A31V':'승합'
}
carr['차종'] = carr['차종'].replace(cd_dict)
carr['차종'].unique()

array(['화물', '승용', '특수', '승합', nan], dtype=object)

## 용도 코드 변환

In [68]:
purps_dict = {
    'A08P':'개인용', 
    'A08B':'영업용', 
    'A08O':'관용',
}
carr['용도'] = carr['용도'].replace(purps_dict)
carr['용도'].unique()

array(['개인용', '영업용', '관용'], dtype=object)

## 등록정보 말소 제거

In [69]:
carm = carr[carr['차량말소YN'] == 'N'].reset_index(drop=True)
carm.shape

(1162311, 10)

## 연료 코드 변환

In [70]:
fuel_dict = {
    'A90GS':'휘발유', 
    'A91DS':'경유',
    'A92LP':'LPG(액화석유가스)', 
    'A90GH':'휘발유 하이브리드', 
    'A93EV':'전기', 
    'A91DH':'경유 하이브리드', 
    'A92CN':'CNG(압축천연가스)', 
    'A93HD':'수소', 
    'A92LH':'LPG 하이브리드', 
    'A94OT':'기타연료', 
    'A92CH':'CNG 하이브리드',
    'A90AC':'알코올', 
    'A93SH':'태양열', 
    'A91KS':'등유', 
    'A92LN':'LNG(액화천연가스)', 
    'A90PH':'플러그인 하이브리드', 
}
srcr['연료'] = srcr['연료'].replace(fuel_dict)
srcr['연료'].unique()

array(['경유', '휘발유', 'CNG(압축천연가스)', nan, 'LPG(액화석유가스)', '기타연료',
       '휘발유 하이브리드', '전기', '알코올', '태양열', '경유 하이브리드', '등유', 'CNG 하이브리드',
       'LPG 하이브리드', '수소', 'LNG(액화천연가스)'], dtype=object)

## 차종유형 코드 변환

In [71]:
ty_dict = {
    'A30C':'경형', 
    'A30L':'대형', 
    'A30M':'중형', 
    'A30S':'소형',
}
srcr['차종유형'] = srcr['차종유형'].replace(ty_dict)
srcr['차종유형'].unique()

array(['대형', '소형', '중형', '경형', nan], dtype=object)

## 등록&제원 병합

In [72]:
# 19.4s
cs = carm.merge(srcr, on='제원관리번호', how='left')
cs.shape

(1162311, 12)

## 저감장치구분 코드 변환

In [73]:
# 4m 14.3s
attr['저감장치구분'].unique()

array(['A1001', 'A1003', 'A1005', 'A1006', 'A1002', 'A1004', 'A1007'],
      dtype=object)

In [74]:
# 27.5s
rdcdvc_dict = {
    'A1001':'1종', 
    'A1002':'2종', 
    'A1003':'3종', 
    'A1004':'1종+SCR', 
    'A1005':'엔진개조', 
    'A1006':'엔진교체',
    'A1007':'삼원촉매',
}
attr['저감장치구분'] = attr['저감장치구분'].replace(rdcdvc_dict)
attr['저감장치구분'].unique()

array(['1종', '3종', '엔진개조', '엔진교체', '2종', '1종+SCR', '삼원촉매'], dtype=object)

## 저감장치 부착 유무

In [75]:
attr.loc[(attr['저감장치구분'] == '1종') | (attr['저감장치구분'] == '1종+SCR'), 'DPF_YN'] = '유'

In [76]:
attr['DPF_YN'].value_counts(dropna=False)

DPF_YN
NaN    531833
유      479113
Name: count, dtype: int64

## 등록&제원&저감이력 병합

In [77]:
# 1.7s
csa = cs.merge(attr[['차대번호', 'DPF_YN']], on='차대번호', how='left')
csa.shape

(1162570, 13)

## 지역정보 병합

In [78]:
csac = csa.merge(coder, on='법정동코드', how='left')

### 매칭 안되는 지역 처리

In [79]:
csac.loc[csac['법정동코드'] == 4165052000, ['시도', '시군구']] = ['경기도', '포천시'] # 경기도 포천시 선단동

## 조기폐차 정보들 병합

In [149]:
elp = pd.concat([aear, lgvr], ignore_index=True)
elp.shape

(1177453, 4)

In [150]:
elp.shape, len(elp['차대번호'].unique())

((1177453, 4), 1105195)

In [151]:
elpm = elp.sort_values('조기폐차최종승인YN', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
elpm.shape

(1105195, 4)

In [152]:
elpm = elpm[elpm['조기폐차최종승인YN'] == 'Y'].reset_index(drop=True)
elpm.shape

(844474, 4)

## 조기폐차 정보 추가

In [153]:
csac.shape

(1162570, 17)

In [154]:
df = csac.merge(elpm, on='차대번호', how='left')
df.shape

(1162570, 20)

In [155]:
df.head()

,차대번호,법정동코드,차량말소YN,제원관리번호,차량연식,차종,용도,최초등록일자,제작일자,차량관리번호,연료,차종유형,DPF_YN,시도코드,시군구코드,시도,시군구,조기폐차상태코드,조기폐차최종승인YN,말소일자
0,KL3C3AHF27K000082,4481025000,N,A0310007202023307,2007,화물,개인용,20220711,20070627.00,KL3C3AHF27K000082-01,경유,대형,NaN,44.00,810.00,충청남도,예산군,NaN,NaN,NaN
1,KL3C3AHF27K000068,4372033032,N,A0310007202023307,2007,화물,개인용,20220413,20070618.00,KL3C3AHF27K000068-01,경유,대형,NaN,43.00,720.00,충청북도,보은군,NaN,NaN,NaN
2,1P3EW65G5XV503775,4157010600,N,10020204100011319,1999,승용,개인용,20211124,20190905,1P3EW65G5XV503775-01,휘발유,대형,NaN,41.00,570.00,경기도,김포시,NaN,NaN,NaN
3,1P3EW65G5YV606177,5011013700,N,10020204100001319,2000,승용,개인용,20211124,20190802,1P3EW65G5YV606177-01,휘발유,대형,NaN,50.00,110.00,제주특별자치도,제주시,NaN,NaN,NaN
4,KMCYSS7JP7U084697,4423038028,N,A0810003403394206,2007,특수,개인용,20210826,20070227,KMCYSS7JP7U084697-01,경유,중형,NaN,44.00,230.00,충청남도,논산시,NaN,NaN,NaN


## 비상시 및 계절제 병합

In [194]:
isdisr.shape, isisr.shape, isper.shape

((81512621, 6), (1785, 3), (10790105, 2))

In [195]:
isdisr.columns, isisr.columns, isper.columns

(Index(['적발번호', '발령번호', '등록시도코드', '등록시군구코드', '적발지역코드', '차대번호'], dtype='object'),
 Index(['발령번호', '농도기준아이디', '유형기준아이디'], dtype='object'),
 Index(['적발번호', '단속일'], dtype='object'))

In [197]:
isdisr.shape

(81512621, 6)

In [198]:
# 41.3s
isdp= isdisr.merge(isper, on='적발번호', how='left')
isdp.shape

(81512621, 7)

In [200]:
# 9.5s
isdpi = isdp.merge(isisr, on='발령번호', how='left')
isdpi.shape

(81512621, 9)

### 지역정보 추가

In [202]:
coder_dup = coder.sort_values('법정동코드', ascending=True).drop_duplicates(['시도코드', '시군구코드']).reset_index(drop=True)

In [203]:
coder_dup.head()

,법정동코드,시도코드,시군구코드,시도,시군구
0,1100000000,11,0,서울특별시,NaN
1,1111000000,11,110,서울특별시,종로구
2,1114000000,11,140,서울특별시,중구
3,1117000000,11,170,서울특별시,용산구
4,1120000000,11,200,서울특별시,성동구


In [411]:
# 28.4s
is_total1 = isdpi.merge(coder_dup[['시도코드', '시군구코드', '시도', '시군구']], left_on=['등록시도코드', '등록시군구코드'], right_on=['시도코드', '시군구코드'], how='left')
is_total1 = is_total1.drop(['시도코드', '시군구코드'], axis=1)
is_total1.shape

(81512621, 11)

In [412]:
is_total1 = is_total1.rename(columns={'시도':'등록시도', '시군구':'등록시군구'})

In [413]:
is_total1.head()

,적발번호,발령번호,등록시도코드,등록시군구코드,적발지역코드,차대번호,단속일,농도기준아이디,유형기준아이디,등록시도,등록시군구
0,54793831,1838,41,190,41190,KMFXKS7BP3U657943,NaN,C012,T001,경기도,부천시
1,54793839,1838,41,190,41190,KNAMA75432S388470,NaN,C012,T001,경기도,부천시
2,54793843,1838,28,237,41190,KNGDNM9N14K178854,NaN,C012,T001,인천광역시,부평구
3,54795717,1838,41,130,41190,KMFGA17PP7C072098,NaN,C012,T001,경기도,성남시
4,54793837,1838,41,190,41190,KPBSH3AB18P082348,NaN,C012,T001,경기도,부천시


In [414]:
# 1m 19.2s
is_total1['적발지역코드'] = is_total1['적발지역코드'].astype('str')
is_total1['적발시도코드'] = is_total1['적발지역코드'].str[:2]
is_total1['적발시군구코드'] = is_total1['적발지역코드'].str[2:5]
is_total1[['적발시도코드', '적발시군구코드']] = is_total1[['적발시도코드', '적발시군구코드']].astype('int')

In [415]:
is_total1.head()

,적발번호,발령번호,등록시도코드,등록시군구코드,적발지역코드,차대번호,단속일,농도기준아이디,유형기준아이디,등록시도,등록시군구,적발시도코드,적발시군구코드
0,54793831,1838,41,190,41190,KMFXKS7BP3U657943,NaN,C012,T001,경기도,부천시,41,190
1,54793839,1838,41,190,41190,KNAMA75432S388470,NaN,C012,T001,경기도,부천시,41,190
2,54793843,1838,28,237,41190,KNGDNM9N14K178854,NaN,C012,T001,인천광역시,부평구,41,190
3,54795717,1838,41,130,41190,KMFGA17PP7C072098,NaN,C012,T001,경기도,성남시,41,190
4,54793837,1838,41,190,41190,KPBSH3AB18P082348,NaN,C012,T001,경기도,부천시,41,190


In [416]:
# 16.5s
is_total = is_total1.merge(coder_dup[['시도코드', '시군구코드', '시도', '시군구']], left_on=['적발시도코드', '적발시군구코드'], right_on=['시도코드', '시군구코드'], how='left')

In [417]:
is_total.head()

,적발번호,발령번호,등록시도코드,등록시군구코드,적발지역코드,차대번호,단속일,농도기준아이디,유형기준아이디,등록시도,등록시군구,적발시도코드,적발시군구코드,시도코드,시군구코드,시도,시군구
0,54793831,1838,41,190,41190,KMFXKS7BP3U657943,NaN,C012,T001,경기도,부천시,41,190,41.00,190.00,경기도,부천시
1,54793839,1838,41,190,41190,KNAMA75432S388470,NaN,C012,T001,경기도,부천시,41,190,41.00,190.00,경기도,부천시
2,54793843,1838,28,237,41190,KNGDNM9N14K178854,NaN,C012,T001,인천광역시,부평구,41,190,41.00,190.00,경기도,부천시
3,54795717,1838,41,130,41190,KMFGA17PP7C072098,NaN,C012,T001,경기도,성남시,41,190,41.00,190.00,경기도,부천시
4,54793837,1838,41,190,41190,KPBSH3AB18P082348,NaN,C012,T001,경기도,부천시,41,190,41.00,190.00,경기도,부천시


In [418]:
# 1m 12.8s
is_total = is_total.drop(['적발시도코드', '적발시군구코드', '시도코드', '시군구코드'], axis=1)
is_total = is_total.rename(columns={'시도':'적발시도', '시군구':'적발시군구'})
is_total.head()

MemoryError: Unable to allocate 622. MiB for an array with shape (1, 81512621) and data type object

## 상시 병합

In [350]:
usdisr.head()

,번호,등록시도코드,등록시군구코드,적발유형,단속지역코드,차대번호
0,117112,41,110,S,41430,KPBFA2AD12P021974
1,117111,11,0,S,41430,KMHNN81XP2U038507
2,117107,41,410,S,41430,KMFZCD7JP5U094049
3,132833,11,0,S,41130,KMFWVH7HP4U579325
4,6933349,11,0,S,41130,KNAMA75433S411400


In [351]:
usper.head()

,번호,적발건수,적발년월
0,8114417,1,201901
1,6933701,4,201901
2,6938411,10,201901
3,8114173,1,201901
4,6938670,14,201901


In [352]:
usdisr.shape

(3090035, 6)

In [353]:
# 1s
usdp = usdisr.merge(usper, on='번호', how='left')
usdp.shape

(3090035, 8)

### 지역정보 추가

In [354]:
us_total = usdp.merge(coder_dup, left_on=['등록시도코드', '등록시군구코드'], right_on=['시도코드', '시군구코드'], how='left')
us_total.head()

,번호,등록시도코드,등록시군구코드,적발유형,단속지역코드,차대번호,적발건수,적발년월,법정동코드,시도코드,시군구코드,시도,시군구
0,117112,41,110,S,41430,KPBFA2AD12P021974,NaN,NaN,4111000000.00,41.00,110.00,경기도,수원시
1,117111,11,0,S,41430,KMHNN81XP2U038507,NaN,NaN,1100000000.00,11.00,0.00,서울특별시,NaN
2,117107,41,410,S,41430,KMFZCD7JP5U094049,NaN,NaN,4141000000.00,41.00,410.00,경기도,군포시
3,132833,11,0,S,41130,KMFWVH7HP4U579325,NaN,NaN,1100000000.00,11.00,0.00,서울특별시,NaN
4,6933349,11,0,S,41130,KNAMA75433S411400,NaN,NaN,1100000000.00,11.00,0.00,서울특별시,NaN


In [355]:
us_total = us_total.drop(['시도코드', '시군구코드'], axis=1)
us_total.head()

,번호,등록시도코드,등록시군구코드,적발유형,단속지역코드,차대번호,적발건수,적발년월,법정동코드,시도,시군구
0,117112,41,110,S,41430,KPBFA2AD12P021974,NaN,NaN,4111000000.00,경기도,수원시
1,117111,11,0,S,41430,KMHNN81XP2U038507,NaN,NaN,1100000000.00,서울특별시,NaN
2,117107,41,410,S,41430,KMFZCD7JP5U094049,NaN,NaN,4141000000.00,경기도,군포시
3,132833,11,0,S,41130,KMFWVH7HP4U579325,NaN,NaN,1100000000.00,서울특별시,NaN
4,6933349,11,0,S,41130,KNAMA75433S411400,NaN,NaN,1100000000.00,서울특별시,NaN


In [356]:
is_total.shape, us_total.shape

((81512621, 13), (3090035, 11))

## 등록(말소 유지) & 제원 병합

In [109]:
# 10.3s
cse = carr.merge(srcr, on='제원관리번호', how='left')

In [110]:
cse.columns

Index(['차대번호', '법정동코드', '차량말소YN', '제원관리번호', '차량연식', '차종', '용도', '최초등록일자',
       '제작일자', '차량관리번호', '연료', '차종유형'],
      dtype='object')

## 등록&제원&이력 병합

In [111]:
# 2m 6.0s
ersr = cse.merge(hisr, on='차량관리번호', how='left')

In [112]:
ersr.columns

Index(['차대번호', '법정동코드', '차량말소YN_x', '제원관리번호', '차량연식', '차종', '용도', '최초등록일자',
       '제작일자', '차량관리번호', '연료', '차종유형', '차량말소YN_y', '변경일자'],
      dtype='object')

In [113]:
# 1. 등록 차량말소와 등록이력 차량 말소 둘 모두 해당되는 데이터 추출
ersr = ersr.loc[(ersr['차량말소YN_x'] == 'Y') & (ersr['차량말소YN_y'] == 'Y')].reset_index(drop=True)

In [114]:
# 2. 변경일자 기준 최신 데이터만 남기고 차대번호 중복 제거
ersr = ersr.sort_values('변경일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)

In [115]:
# 3. 변경일자 2019.01.01 이후만 추출
ersr = ersr[ersr['변경일자'] >= 20190101].reset_index(drop=True)

## 등록&제원&이력&법정동코드 병합(errc)

In [116]:
# 매칭 후 '시도' 빈값 없음
errc = ersr.merge(coder, on='법정동코드', how='left')

# Load

## 5등급 지역별 조기폐차

In [117]:
# about 2.1s
name = '5등급_조기폐차'
file_name = f'{name}.xlsx'
df1 = pd.read_excel(os.path.join(raw3_fold, file_name))
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   RGN         68 non-null     object 
 1   YR          68 non-null     int64  
 2   VHCL_MKCNT  68 non-null     int64  
 3   VHCL_REDE   68 non-null     float64
 4   ELPDSRC     68 non-null     float64
 5   NTRL_DCLN   68 non-null     float64
dtypes: float64(3), int64(2), object(1)
memory usage: 3.3+ KB


In [118]:
df1.head()

,RGN,YR,VHCL_MKCNT,VHCL_REDE,ELPDSRC,NTRL_DCLN
0,서울,2019,199898,46949.67,31792.00,15157.67
1,서울,2020,161864,38034.00,23045.00,14989.00
2,서울,2021,138381,23483.00,8148.00,15335.00
3,서울,2022,120996,17385.00,2476.00,14909.00
4,인천,2019,90619,20596.67,16421.67,4175.00


## 5등급 저공해 미조치

In [119]:
# about 2.1s
name = '5등급_저공해미조치'
file_name = f'{name}.xlsx'
no_dpf = pd.read_excel(os.path.join(raw3_fold, file_name))
no_dpf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   지역      12 non-null     object
 1   구분      12 non-null     object
 2   차량대수    12 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 416.0+ bytes


In [120]:
no_dpf.head()

,지역,구분,차량대수
0,수도권,2차계절제_시작,539748
1,수도권,2차계절제_종료,515615
2,수도권,3차계절제_시작,459110
3,수도권,3차계절제_종료,436921
4,수도권,4차계절제_시작,394533


## [로드] 운행제한 테이블(limit)

In [308]:
# about 2.1s
name = '차대번호별 적발'
file_name = f'{name}.csv'
limit = pd.read_csv(os.path.join(raw3_fold, file_name), low_memory=False, encoding='cp949')
limit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731507 entries, 0 to 731506
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   VIN     731507 non-null  object 
 1   계절제_1차  37861 non-null   float64
 2   계절제_2차  54788 non-null   float64
 3   계절제_3차  68278 non-null   float64
 4   비상시     94084 non-null   float64
 5   상시      84185 non-null   float64
 6   계절제_4차  29307 non-null   float64
dtypes: float64(6), object(1)
memory usage: 39.1+ MB


In [309]:
limit.head()

,VIN,계절제_1차,계절제_2차,계절제_3차,비상시,상시,계절제_4차
0,KPBFA2AD14P100772,1.00,NaN,NaN,1.00,NaN,NaN
1,KMJWWH7JP5U656738,4.00,2.00,25.00,3.00,NaN,1.00
2,KNAJC52136A532787,2.00,NaN,NaN,1.00,NaN,NaN
3,KMHSH81WP6U081899,4.00,NaN,NaN,1.00,NaN,NaN
4,KNAJE55186K143510,1.00,NaN,NaN,1.00,NaN,NaN


In [310]:
limit['상시'].describe()

count   84185.00
mean        5.27
std         5.66
min         1.00
25%         1.00
50%         4.00
75%         7.00
max        58.00
Name: 상시, dtype: float64

## [로드] 계절제 적발지역별 차대번호(slimit)

In [123]:
# about 1.8s
name = '계절제 적발지역 차대번호'
file_name = f'{name}.csv'
slimit = pd.read_csv(os.path.join(raw3_fold, file_name), low_memory=False, encoding='cp949')
slimit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163294 entries, 0 to 163293
Data columns (total 9 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   적발지역    163294 non-null  object 
 1   적발시도    163294 non-null  object 
 2   차대번호    163294 non-null  object 
 3   계절제_1차  39111 non-null   float64
 4   계절제_2차  58088 non-null   float64
 5   계절제_3차  73775 non-null   float64
 6   계절제_4차  31673 non-null   float64
 7   차종      162378 non-null  object 
 8   차종유형    162378 non-null  object 
dtypes: float64(4), object(5)
memory usage: 11.2+ MB


# 분석

In [125]:
us_total.loc[us_total['적발유형'] == 'H'].groupby(['시도'])['적발건수'].sum()

Series([], Name: 적발건수, dtype: float64)

## 5등급

### 5등급 지역별 조기폐차 현황

In [156]:
df.head()

,차대번호,법정동코드,차량말소YN,제원관리번호,차량연식,차종,용도,최초등록일자,제작일자,차량관리번호,연료,차종유형,DPF_YN,시도코드,시군구코드,시도,시군구,조기폐차상태코드,조기폐차최종승인YN,말소일자
0,KL3C3AHF27K000082,4481025000,N,A0310007202023307,2007,화물,개인용,20220711,20070627.00,KL3C3AHF27K000082-01,경유,대형,NaN,44.00,810.00,충청남도,예산군,NaN,NaN,NaN
1,KL3C3AHF27K000068,4372033032,N,A0310007202023307,2007,화물,개인용,20220413,20070618.00,KL3C3AHF27K000068-01,경유,대형,NaN,43.00,720.00,충청북도,보은군,NaN,NaN,NaN
2,1P3EW65G5XV503775,4157010600,N,10020204100011319,1999,승용,개인용,20211124,20190905,1P3EW65G5XV503775-01,휘발유,대형,NaN,41.00,570.00,경기도,김포시,NaN,NaN,NaN
3,1P3EW65G5YV606177,5011013700,N,10020204100001319,2000,승용,개인용,20211124,20190802,1P3EW65G5YV606177-01,휘발유,대형,NaN,50.00,110.00,제주특별자치도,제주시,NaN,NaN,NaN
4,KMCYSS7JP7U084697,4423038028,N,A0810003403394206,2007,특수,개인용,20210826,20070227,KMCYSS7JP7U084697-01,경유,중형,NaN,44.00,230.00,충청남도,논산시,NaN,NaN,NaN


In [157]:
errc.head()

,차대번호,법정동코드,차량말소YN_x,제원관리번호,차량연식,차종,용도,최초등록일자,제작일자,차량관리번호,...,차종유형,차량말소YN_y,변경일자,시도코드,시군구코드,시도,시군구,변경일자_년,변경일자_월,변경일자_일
0,440234211982,2811013800,Y,10172800010000,2002,화물,관용,20020318,20020313.00,440234211982-01,...,중형,Y,20220628.0,28,110,인천광역시,중구,2022,06,28
1,KNAJC52183A097253,4825037023,Y,A0110002000041302,2003,승용,개인용,20030109,20030104,KNAJC52183A097253-01,...,대형,Y,20220628.0,48,250,경상남도,김해시,2022,06,28
2,KNCSC9342PS667181,2635010100,Y,10018600430000,1993,화물,개인용,19930727,NaN,KNCSC9342PS667181-01,...,소형,Y,20220628.0,26,350,부산광역시,해운대구,2022,06,28
3,KMJWWH7JP3U533880,4725034030,Y,A0810003700112202,2003,승합,개인용,20030408,20030318.00,KMJWWH7JP3U533880-01,...,중형,Y,20220628.0,47,250,경상북도,상주시,2022,06,28
4,KNAJC52184A284285,3011011400,Y,A0110002001621303,2004,승용,개인용,20040308,20040225,KNAJC52184A284285-01,...,대형,Y,20220628.0,30,110,대전광역시,동구,2022,06,28


In [158]:
dfm = df.copy()

In [159]:
dfm['최초등록일자'] = dfm['최초등록일자'].astype('str')
dfm['최초등록일자_년'] = dfm['최초등록일자'].str[:4]
dfm['최초등록일자_월'] = dfm['최초등록일자'].str[4:6]
dfm['최초등록일자_일'] = dfm['최초등록일자'].str[6:8]

In [130]:
errc['변경일자'] = errc['변경일자'].astype('str')
errc['변경일자_년'] = errc['변경일자'].str[:4]
errc['변경일자_월'] = errc['변경일자'].str[4:6]
errc['변경일자_일'] = errc['변경일자'].str[6:8]

### 시도, 연도별 차량 현황 분석

In [160]:
# 2022년 차량 대수
grp1 = dfm.groupby(['시도'], as_index=False)['차대번호'].count()
grp1 = grp1.rename(columns={'차대번호':'차량대수'})
grp1['연도'] = '2022'
grp1 = grp1[['연도', '시도', '차량대수']]

In [132]:
grp1.head()

,연도,시도,차량대수
0,2022,강원도,48918
1,2022,경기도,229975
2,2022,경상남도,105851
3,2022,경상북도,123303
4,2022,광주광역시,26496


In [133]:
# 4년간 차량 통계 기본 데이터셋
yr_list = []
ctpv_list = []
for ctpv in grp1['시도'].unique():
    for yr in ['2019', '2020', '2021', '2022']:
        yr_list.append(yr)
        ctpv_list.append(ctpv)
base = pd.DataFrame({'연도':yr_list, '시도':ctpv_list})

In [134]:
base.head()

,연도,시도
0,2019,강원도
1,2020,강원도
2,2021,강원도
3,2022,강원도
4,2019,경기도


In [135]:
# 연도별 등록대수
grp2 = dfm.groupby(['최초등록일자_년', '시도'], as_index=False)['차대번호'].count()
grp2 = grp2.rename(columns={'최초등록일자_년':'연도', '차대번호':'등록대수'})
grp2

,연도,시도,등록대수
0,1950,경기도,1
1,1955,강원도,1
2,1958,경상북도,1
3,1962,강원도,4
4,1962,경기도,2
...,...,...,...
812,2021,울산광역시,1
813,2021,제주특별자치도,1
814,2021,충청남도,2
815,2022,충청남도,1


In [136]:
# 연도별 말소대수
grp3 = errc.groupby(['변경일자_년', '시도'], as_index=False)['차대번호'].count()
grp3 = grp3.rename(columns={'변경일자_년':'연도', '차대번호':'말소대수'})
grp3

,연도,시도,말소대수
0,2019,강원도,11881
1,2019,경기도,73287
2,2019,경상남도,17542
3,2019,경상북도,19636
4,2019,광주광역시,5895
...,...,...,...
63,2022,전라남도,11191
64,2022,전라북도,6653
65,2022,제주특별자치도,3391
66,2022,충청남도,8951


In [137]:
base1 = base.merge(grp1, on=['연도', '시도'], how='left')
base1

,연도,시도,차량대수
0,2019,강원도,NaN
1,2020,강원도,NaN
2,2021,강원도,NaN
3,2022,강원도,48918.00
4,2019,경기도,NaN
...,...,...,...
63,2022,충청남도,63513.00
64,2019,충청북도,NaN
65,2020,충청북도,NaN
66,2021,충청북도,NaN


In [138]:
base2 = base1.merge(grp2, on=['연도', '시도'], how='left')
base2

,연도,시도,차량대수,등록대수
0,2019,강원도,NaN,2.00
1,2020,강원도,NaN,1.00
2,2021,강원도,NaN,NaN
3,2022,강원도,48918.00,NaN
4,2019,경기도,NaN,36.00
...,...,...,...,...
63,2022,충청남도,63513.00,1.00
64,2019,충청북도,NaN,2.00
65,2020,충청북도,NaN,NaN
66,2021,충청북도,NaN,NaN


In [139]:
base3 = base2.merge(grp3, on=['연도', '시도'], how='left')
base3

,연도,시도,차량대수,등록대수,말소대수
0,2019,강원도,NaN,2.00,11881
1,2020,강원도,NaN,1.00,16689
2,2021,강원도,NaN,NaN,19202
3,2022,강원도,48918.00,NaN,6192
4,2019,경기도,NaN,36.00,73287
...,...,...,...,...,...
63,2022,충청남도,63513.00,1.00,8951
64,2019,충청북도,NaN,2.00,13677
65,2020,충청북도,NaN,NaN,23589
66,2021,충청북도,NaN,NaN,19555


In [140]:
base3[['차량대수', '등록대수', '말소대수']].isnull().sum()

차량대수    51
등록대수    37
말소대수     0
dtype: int64

In [141]:
base3[['차량대수', '등록대수', '말소대수']] = base3[['차량대수', '등록대수', '말소대수']].fillna(0)

In [142]:
n = 4
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소대수'] - base3.loc[(i+1)*n - (j-1), '등록대수']

In [143]:
base3

,연도,시도,차량대수,등록대수,말소대수
0,2019,강원도,91000.00,2.00,11881
1,2020,강원도,74312.00,1.00,16689
2,2021,강원도,55110.00,0.00,19202
3,2022,강원도,48918.00,0.00,6192
4,2019,경기도,405494.00,36.00,73287
...,...,...,...,...,...
63,2022,충청남도,63513.00,1.00,8951
64,2019,충청북도,90869.00,2.00,13677
65,2020,충청북도,67280.00,0.00,23589
66,2021,충청북도,47725.00,0.00,19555


In [163]:
dfm.columns

Index(['차대번호', '법정동코드', '차량말소YN', '제원관리번호', '차량연식', '차종', '용도', '최초등록일자',
       '제작일자', '차량관리번호', '연료', '차종유형', 'DPF_YN', '시도코드', '시군구코드', '시도', '시군구',
       '조기폐차상태코드', '조기폐차최종승인YN', '말소일자', '최초등록일자_년', '최초등록일자_월', '최초등록일자_일'],
      dtype='object')

In [165]:
dfm['말소일자'] = dfm['말소일자'].astype('str')
dfm['말소일자_년'] = dfm['말소일자'].str[:4]
dfm['말소일자_월'] = dfm['말소일자'].str[4:6]
dfm['말소일자_일'] = dfm['말소일자'].str[6:8]

In [168]:
grp4 = dfm.loc[dfm['조기폐차최종승인YN'] == 'Y'].groupby(['말소일자_년'], as_index=False)['차대번호'].count()
grp4 = grp4.rename(columns={'말소일자_년':'연도', '차대번호':'조기폐차대수'})
grp4

,연도,조기폐차대수
0,2013,1
1,2015,1
2,2018,1
3,2021,2
4,2022,25147
5,2023,15953
6,2032,4
7,2202,5
8,nan,1


In [169]:
base4 = base3.merge(grp4, on='연도', how='left')
base4

,연도,시도,차량대수,등록대수,말소대수,조기폐차대수
0,2019,강원도,91000.00,2.00,11881,NaN
1,2020,강원도,74312.00,1.00,16689,NaN
2,2021,강원도,55110.00,0.00,19202,2.00
3,2022,강원도,48918.00,0.00,6192,25147.00
4,2019,경기도,405494.00,36.00,73287,NaN
...,...,...,...,...,...,...
63,2022,충청남도,63513.00,1.00,8951,25147.00
64,2019,충청북도,90869.00,2.00,13677,NaN
65,2020,충청북도,67280.00,0.00,23589,NaN
66,2021,충청북도,47725.00,0.00,19555,2.00


In [172]:
base4['조기폐차대수'] = base4['조기폐차대수'].fillna(0)
base4 = base4.drop(['등록대수', '말소대수'], axis=1)
base4

,연도,시도,차량대수,조기폐차대수
0,2019,강원도,91000.00,0.00
1,2020,강원도,74312.00,0.00
2,2021,강원도,55110.00,2.00
3,2022,강원도,48918.00,25147.00
4,2019,경기도,405494.00,0.00
...,...,...,...,...
63,2022,충청남도,63513.00,25147.00
64,2019,충청북도,90869.00,0.00
65,2020,충청북도,67280.00,0.00
66,2021,충청북도,47725.00,2.00


In [174]:
n = len(base4['연도'].unique())
for i in range(base4.shape[0] // n):
    for j in range(n-1):
        base4.loc[i*4 + j+1, '감소대수'] = base4.loc[i*4 + j, '차량대수'] - base4.loc[i*4 + j+1, '차량대수']

In [175]:
base4

,연도,시도,차량대수,조기폐차대수,감소대수
0,2019,강원도,91000.00,0.00,NaN
1,2020,강원도,74312.00,0.00,16688.00
2,2021,강원도,55110.00,2.00,19202.00
3,2022,강원도,48918.00,25147.00,6192.00
4,2019,경기도,405494.00,0.00,NaN
...,...,...,...,...,...
63,2022,충청남도,63513.00,25147.00,8950.00
64,2019,충청북도,90869.00,0.00,NaN
65,2020,충청북도,67280.00,0.00,23589.00
66,2021,충청북도,47725.00,2.00,19555.00


In [176]:
base4['자연감소대수'] = base4['감소대수'] - base4['조기폐차대수']

In [179]:
base4

,연도,시도,차량대수,조기폐차대수,감소대수,자연감소대수
0,2019,강원도,91000.00,0.00,NaN,NaN
1,2020,강원도,74312.00,0.00,16688.00,16688.00
2,2021,강원도,55110.00,2.00,19202.00,19200.00
3,2022,강원도,48918.00,25147.00,6192.00,-18955.00
4,2019,경기도,405494.00,0.00,NaN,NaN
...,...,...,...,...,...,...
63,2022,충청남도,63513.00,25147.00,8950.00,-16197.00
64,2019,충청북도,90869.00,0.00,NaN,NaN
65,2020,충청북도,67280.00,0.00,23589.00,23589.00
66,2021,충청북도,47725.00,2.00,19555.00,19553.00


In [9]:
STD_BD_GRD5_ELPDSRC_CURSTT = df1.copy()

#### [출력] [D] STD_BD_GRD5_ELPDSRC_CURSTT

In [ ]:
# expdf = STD_BD_GRD5_ELPDSRC_CURSTT
# table_nm = 'STD_BD_GRD5_ELPDSRC_CURSTT'.upper()

# # 테이블 생성
# sql = 'create table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# wbt.execute(sql)

# # 데이터 추가
# # 5s
# wbt.import_from_pandas(expdf, table_nm)

### 5등급 지역별 저공해미조치 차량현황

In [10]:
cdict = {
    '지역':'RGN',
    '구분':'SEASON',
    '차량대수':'VHCL_MKCNT', 
}

In [11]:
STD_BD_GRD5_LEM_N_MOD = no_dpf.rename(columns=cdict)
STD_BD_GRD5_LEM_N_MOD.columns

Index(['RGN', 'SEASON', 'VHCL_MKCNT'], dtype='object')

#### [출력] [D] STD_BD_GRD5_LEM_N_MOD

In [ ]:
# expdf = STD_BD_GRD5_LEM_N_MOD
# table_nm = 'STD_BD_GRD5_LEM_N_MOD'.upper()

# # 테이블 생성
# sql = 'create table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# wbt.execute(sql)

# # 데이터 추가
# # 5s
# wbt.import_from_pandas(expdf, table_nm)

### 차대번호별 운행제한 적발 현황

In [ ]:
# DNSTY_STDR_ID(농도기준아이디) : 실발령(C011), 모의발령(C012)
# TY_STDR_ID(유형기준아이디) : 비상시(T001), 계절제(T002)

In [215]:
is_total.head()

,적발번호,발령번호,등록시도코드,등록시군구코드,적발지역코드,차대번호,단속일,농도기준아이디,유형기준아이디,등록시도,등록시군구,적발시도,적발시도
0,54793831,1838,41,190,41190,KMFXKS7BP3U657943,NaN,C012,T001,경기도,부천시,경기도,부천시
1,54793839,1838,41,190,41190,KNAMA75432S388470,NaN,C012,T001,경기도,부천시,경기도,부천시
2,54793843,1838,28,237,41190,KNGDNM9N14K178854,NaN,C012,T001,인천광역시,부평구,경기도,부천시
3,54795717,1838,41,130,41190,KMFGA17PP7C072098,NaN,C012,T001,경기도,성남시,경기도,부천시
4,54793837,1838,41,190,41190,KPBSH3AB18P082348,NaN,C012,T001,경기도,부천시,경기도,부천시


In [216]:
is_total.shape

(81512621, 13)

In [311]:
# 11.0s
# DNSTY_STDR_ID(농도기준아이디) : 실발령(C011), 모의발령(C012)
# TY_STDR_ID(유형기준아이디) : 비상시(T001), 계절제(T002)
is_season = is_total.loc[(is_total['농도기준아이디'] == 'C011') & (is_total['유형기준아이디'] == 'T002')].reset_index(drop=True)
is_season.shape

(13012643, 13)

In [312]:
is_season['단속일'].dtype

dtype('float64')

In [313]:
is_season['단속일'].describe()

count     338694.00
mean    20211753.01
std         7033.04
min     20201201.00
25%     20210112.00
50%     20210316.00
75%     20220121.00
max     20230104.00
Name: 단속일, dtype: float64

In [314]:
is_season.loc[(is_season['단속일'] > 20191130) & (is_season['단속일'] < 20200401), '계절제_1차여부'] = 'Y'
is_season.loc[(is_season['단속일'] > 20201130) & (is_season['단속일'] < 20210401), '계절제_2차여부'] = 'Y'
is_season.loc[(is_season['단속일'] > 20211130) & (is_season['단속일'] < 20220401), '계절제_3차여부'] = 'Y'
is_season.loc[(is_season['단속일'] > 20221130) & (is_season['단속일'] < 20230401), '계절제_4차여부'] = 'Y'

In [315]:
limit_season = is_season.groupby(['차대번호'], as_index=False).agg({'계절제_1차여부':'count', '계절제_2차여부':'count', '계절제_3차여부':'count', '계절제_4차여부':'count'})
limit_season = limit_season.rename(columns={'계절제_1차여부':'계절제_1차', '계절제_2차여부':'계절제_2차', '계절제_3차여부':'계절제_3차', '계절제_4차여부':'계절제_4차'})
limit_season

,차대번호,계절제_1차,계절제_2차,계절제_3차,계절제_4차
0,000000JN44C000673,0,0,9,0
1,01451201380,0,0,0,0
2,01751300580,0,1,0,0
3,01761300921,0,0,0,0
4,01761301128,0,0,1,0
...,...,...,...,...,...
408191,YV2JSO2C668870647,0,1,0,0
408192,YV2JSO2C768870642,0,0,1,2
408193,YV2JSO2C76A625817,0,0,0,0
408194,YV2JSO2C88A655606,0,0,0,0


In [316]:
# 11.0s
# DNSTY_STDR_ID(농도기준아이디) : 실발령(C011), 모의발령(C012)
# TY_STDR_ID(유형기준아이디) : 비상시(T001), 계절제(T002)
is_high = is_total.loc[(is_total['농도기준아이디'] == 'C011') & (is_total['유형기준아이디'] == 'T001')].reset_index(drop=True)
is_high.shape

(546574, 13)

In [317]:
is_high.head()

,적발번호,발령번호,등록시도코드,등록시군구코드,적발지역코드,차대번호,단속일,농도기준아이디,유형기준아이디,등록시도,등록시군구,적발시도,적발시도
0,19890,1,11,350,41150,KMHMG81VP5U257794,NaN,C011,T001,서울특별시,노원구,경기도,의정부시
1,39000,1,41,210,41210,KNAMC75136S062407,NaN,C011,T001,경기도,광명시,경기도,광명시
2,19892,1,41,360,41360,KNAJE55136K140658,NaN,C011,T001,경기도,남양주시,경기도,남양주시
3,19889,1,41,610,41130,KNAJC52134A283624,NaN,C011,T001,경기도,광주시,경기도,성남시
4,19191,1,11,500,41190,KN4HNW6DM5K098136,NaN,C011,T001,서울특별시,강서구,경기도,부천시


In [318]:
limit_high = is_high.groupby(['차대번호'], as_index=False).agg({'단속일':'count'})
limit_high = limit_high.rename(columns={'단속일':'비상시'})
limit_high

,차대번호,비상시
0,000000JN44C000673,1
1,01841300288,1
2,01861321189,0
3,02821282638,2
4,03824219227,1
...,...,...
188457,YV2JS02CX8A655591,0
188458,YV2JSO2C768870642,1
188459,YV2JSO2C76A625817,1
188460,YV2JSO2C88A655606,0


In [319]:
us_total.shape

(3090035, 10)

In [320]:
us_total['적발건수'].dtype

dtype('float64')

In [321]:
limit_alw = us_total.groupby('차대번호', as_index=False).agg({'적발건수':'sum'})
limit_alw = limit_alw.rename(columns={'적발건수':'상시'})
limit_alw

,차대번호,상시
0,01771226923,3.00
1,01781201391,1.00
2,01781212746,1.00
3,01791201714,2.00
4,01792116793,7.00
...,...,...
85750,YV2JSO2C668870647,64.00
85751,YV2JSO2C768870642,4.00
85752,YV2JSO2C76A620052,1.00
85753,YV2JSO2C968870643,86.00


In [322]:
limit_season.shape, limit_high.shape, limit_alw.shape

((408196, 5), (188462, 2), (85755, 2))

In [323]:
limit_sh = limit_season.merge(limit_high, on='차대번호', how='left')
limit_sh.shape

(408196, 6)

In [324]:
limit = limit_sh.merge(limit_alw, on='차대번호', how='left')
limit.shape

(408196, 7)

In [325]:
limit.columns

Index(['차대번호', '계절제_1차', '계절제_2차', '계절제_3차', '계절제_4차', '비상시', '상시'], dtype='object')

In [326]:
limit[['계절제_1차', '계절제_2차', '계절제_3차', '계절제_4차', '비상시', '상시']].isnull().sum()

계절제_1차         0
계절제_2차         0
계절제_3차         0
계절제_4차         0
비상시       271900
상시        360710
dtype: int64

In [327]:
limit[['계절제_1차', '계절제_2차', '계절제_3차', '계절제_4차', '비상시', '상시']] = limit[['계절제_1차', '계절제_2차', '계절제_3차', '계절제_4차', '비상시', '상시']].fillna(0)
limit[['계절제_1차', '계절제_2차', '계절제_3차', '계절제_4차', '비상시', '상시']].isnull().sum()

계절제_1차    0
계절제_2차    0
계절제_3차    0
계절제_4차    0
비상시       0
상시        0
dtype: int64

In [328]:
limit['비상시'] = limit['비상시'].astype('int')
limit['상시'] = limit['상시'].astype('int')

In [329]:
limit.head()

,차대번호,계절제_1차,계절제_2차,계절제_3차,계절제_4차,비상시,상시
0,000000JN44C000673,0,0,9,0,1,0
1,01451201380,0,0,0,0,0,0
2,01751300580,0,1,0,0,0,0
3,01761300921,0,0,0,0,0,0
4,01761301128,0,0,1,0,0,0


In [330]:
limit['상시'].describe()

count   408196.00
mean         3.50
std         20.33
min          0.00
25%          0.00
50%          0.00
75%          0.00
max        900.00
Name: 상시, dtype: float64

In [331]:
df.shape

(1162570, 20)

In [332]:
lmt1 = df.merge(limit, on='차대번호', how='left')
lmt1.shape

(1162570, 26)

In [333]:
lmt1.columns

Index(['차대번호', '법정동코드', '차량말소YN', '제원관리번호', '차량연식', '차종', '용도', '최초등록일자',
       '제작일자', '차량관리번호', '연료', '차종유형', 'DPF_YN', '시도코드', '시군구코드', '시도', '시군구',
       '조기폐차상태코드', '조기폐차최종승인YN', '말소일자', '계절제_1차', '계절제_2차', '계절제_3차',
       '계절제_4차', '비상시', '상시'],
      dtype='object')

In [334]:
lmt1['시도'].unique()

array(['충청남도', '충청북도', '경기도', '제주특별자치도', '경상북도', '울산광역시', '경상남도', '광주광역시',
       '부산광역시', '대전광역시', '전라남도', '강원도', '대구광역시', '전라북도', '서울특별시', '인천광역시',
       '세종특별자치시'], dtype=object)

In [335]:
lmt1.loc[(lmt1['시도'] == '서울특별시') | (lmt1['시도'] == '경기도') | (lmt1['시도'] == '인천광역시'), '지역'] = '수도권'
lmt1['지역'] = lmt1['지역'].fillna('수도권외')

In [336]:
lmt1['지역'].unique()

array(['수도권외', '수도권'], dtype=object)

In [337]:
lmt1['DPF_YN'].value_counts(dropna=False)

DPF_YN
NaN    843374
유      319196
Name: count, dtype: int64

In [338]:
lmt1['DPF_YN'] = lmt1['DPF_YN'].fillna('무')

In [339]:
lmt1.columns

Index(['차대번호', '법정동코드', '차량말소YN', '제원관리번호', '차량연식', '차종', '용도', '최초등록일자',
       '제작일자', '차량관리번호', '연료', '차종유형', 'DPF_YN', '시도코드', '시군구코드', '시도', '시군구',
       '조기폐차상태코드', '조기폐차최종승인YN', '말소일자', '계절제_1차', '계절제_2차', '계절제_3차',
       '계절제_4차', '비상시', '상시', '지역'],
      dtype='object')

In [340]:
season = lmt1.loc[ (lmt1['계절제_1차'] > 0) | (lmt1['계절제_2차'] > 0) | (lmt1['계절제_3차'] > 0) | (lmt1['계절제_4차'] > 0),[
    '차대번호', 
    '계절제_1차', 
    '계절제_2차',
    '계절제_3차', 
    '계절제_4차',
    '지역', 
    '시도', 
    'DPF_YN', 
    '차종', 
    '차종유형', 
]]
season.shape

(54824, 10)

In [341]:
cdict = {
    '차대번호':'VIN', 
    '계절제_1차':'SEASON_1ODR_CRDN_NOCS', 
    '계절제_2차':'SEASON_2ODR_CRDN_NOCS',
    '계절제_3차':'SEASON_3ODR_CRDN_NOCS', 
    '계절제_4차':'SEASON_4ODR_CRDN_NOCS',
    '지역':'RGN', 
    '시도':'CTPV', 
    'DPF_YN':'DPF_EXTRNS_YN', 
    '차종':'VHCTY_CD', 
    '차종유형':'VHCTY_TY', 
}

In [342]:
STD_BD_SEASON_CRDN_NOCS_CURSTT = season.rename(columns=cdict)
STD_BD_SEASON_CRDN_NOCS_CURSTT.columns

Index(['VIN', 'SEASON_1ODR_CRDN_NOCS', 'SEASON_2ODR_CRDN_NOCS',
       'SEASON_3ODR_CRDN_NOCS', 'SEASON_4ODR_CRDN_NOCS', 'RGN', 'CTPV',
       'DPF_EXTRNS_YN', 'VHCTY_CD', 'VHCTY_TY'],
      dtype='object')

#### [출력] STD_BD_SEASON_CRDN_NOCS_CURSTT

In [ ]:
# expdf = STD_BD_SEASON_CRDN_NOCS_CURSTT
# table_nm = 'STD_BD_SEASON_CRDN_NOCS_CURSTT'.upper()

# # 테이블 생성
# sql = 'create table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# wbt.execute(sql)

# # 데이터 추가
# # 5s
# wbt.import_from_pandas(expdf, table_nm)

### 지역별 상시운행제한 단속 현황

In [369]:
us_total.shape

(3090035, 12)

In [396]:
us_total2 = us_total.merge(limit_alw, on='차대번호', how='left')
us_total2.shape

(3090035, 13)

In [397]:
us_total2.head()

,번호,등록시도코드,등록시군구코드,적발유형,단속지역코드,차대번호,적발건수,적발년월,법정동코드,시도,시군구,적발년월_년,상시
0,117112,41,110,S,41430,KPBFA2AD12P021974,NaN,nan,4111000000.00,경기도,수원시,nan,0.00
1,117111,11,0,S,41430,KMHNN81XP2U038507,NaN,nan,1100000000.00,서울특별시,NaN,nan,78.00
2,117107,41,410,S,41430,KMFZCD7JP5U094049,NaN,nan,4141000000.00,경기도,군포시,nan,0.00
3,132833,11,0,S,41130,KMFWVH7HP4U579325,NaN,nan,1100000000.00,서울특별시,NaN,nan,12.00
4,6933349,11,0,S,41130,KNAMA75433S411400,NaN,nan,1100000000.00,서울특별시,NaN,nan,211.00


In [398]:
us_total2.loc[(us_total2['시도'] == '서울특별시') | (us_total2['시도'] == '경기도') | (us_total2['시도'] == '인천광역시'), '지역'] = '수도권'
us_total2['지역'] = us_total2['지역'].fillna('수도권외')

In [399]:
us_total2 = us_total2.sort_values('적발년월_년', ascending=True).drop_duplicates('차대번호').reset_index(drop=True)
us_total2.shape

(85755, 14)

In [400]:
us_total2.head()

,번호,등록시도코드,등록시군구코드,적발유형,단속지역코드,차대번호,적발건수,적발년월,법정동코드,시도,시군구,적발년월_년,상시,지역
0,29292052,11,140,S,11000,KMHWP81HP4U595502,1.00,201910.0,1114000000.00,서울특별시,중구,2019,21.00,수도권
1,27704334,48,330,S,11000,KL3XD70C1TK000480,2.00,201909.0,4833000000.00,경상남도,양산시,2019,14.00,수도권외
2,7988381,11,140,S,11000,KMHMH81VP5U260393,4.00,201905.0,1114000000.00,서울특별시,중구,2019,99.00,수도권
3,27704294,11,140,S,11000,KMFFD37BPXU433184,1.00,201909.0,1114000000.00,서울특별시,중구,2019,1.00,수도권
4,7988382,11,140,S,11000,KNAMA75434S616088,8.00,201905.0,1114000000.00,서울특별시,중구,2019,64.00,수도권


In [401]:
us_total2 = us_total2.drop(['적발건수'], axis=1)
us_total2.head()

,번호,등록시도코드,등록시군구코드,적발유형,단속지역코드,차대번호,적발년월,법정동코드,시도,시군구,적발년월_년,상시,지역
0,29292052,11,140,S,11000,KMHWP81HP4U595502,201910.0,1114000000.00,서울특별시,중구,2019,21.00,수도권
1,27704334,48,330,S,11000,KL3XD70C1TK000480,201909.0,4833000000.00,경상남도,양산시,2019,14.00,수도권외
2,7988381,11,140,S,11000,KMHMH81VP5U260393,201905.0,1114000000.00,서울특별시,중구,2019,99.00,수도권
3,27704294,11,140,S,11000,KMFFD37BPXU433184,201909.0,1114000000.00,서울특별시,중구,2019,1.00,수도권
4,7988382,11,140,S,11000,KNAMA75434S616088,201905.0,1114000000.00,서울특별시,중구,2019,64.00,수도권


In [403]:
us_total2 = us_total2.rename(columns={'적발년월_년':'적발년도', '상시':'적발건수'})
us_total2.head()

,번호,등록시도코드,등록시군구코드,적발유형,단속지역코드,차대번호,적발년월,법정동코드,시도,시군구,적발년도,적발건수,지역
0,29292052,11,140,S,11000,KMHWP81HP4U595502,201910.0,1114000000.00,서울특별시,중구,2019,21.00,수도권
1,27704334,48,330,S,11000,KL3XD70C1TK000480,201909.0,4833000000.00,경상남도,양산시,2019,14.00,수도권외
2,7988381,11,140,S,11000,KMHMH81VP5U260393,201905.0,1114000000.00,서울특별시,중구,2019,99.00,수도권
3,27704294,11,140,S,11000,KMFFD37BPXU433184,201909.0,1114000000.00,서울특별시,중구,2019,1.00,수도권
4,7988382,11,140,S,11000,KNAMA75434S616088,201905.0,1114000000.00,서울특별시,중구,2019,64.00,수도권


In [404]:
us_total2['적발건수'].dtype

dtype('float64')

In [405]:
orditm = us_total2.loc[(us_total2['적발건수'] > 0)& (us_total2['적발년도'].isnull() == False), [
    '차대번호',
    '적발년도',
    '적발건수',
    '지역',
    '시도',
]]

In [406]:
orditm.shape

(83631, 5)

In [407]:
orditm['적발건수'].describe()

count   83631.00
mean       23.77
std        44.56
min         1.00
25%         2.00
50%         8.00
75%        27.00
max       900.00
Name: 적발건수, dtype: float64

#### [❌] [로드] 상시 적발지역 차대번호2

In [278]:
# # about 2.1s
# name = '상시 적발지역 차대번호2'
# file_name = f'{name}.csv'
# orditm = pd.read_csv(os.path.join(raw3_fold, file_name), encoding='cp949')
# orditm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106472 entries, 0 to 106471
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   차대번호    106472 non-null  object
 1   적발년도    106472 non-null  int64 
 2   적발건수    106472 non-null  int64 
 3   지역      106472 non-null  object
 4   시도      106472 non-null  object
dtypes: int64(2), object(3)
memory usage: 4.1+ MB


In [280]:
# orditm[orditm['적발건수'] > 0]

,차대번호,적발년도,적발건수,지역,시도
0,KNGDPM5313K123343,2019,2,수도권,서울
1,KNGDPM5N14K163032,2019,4,수도권,서울
2,KNHMB76136S072660,2019,4,수도권,인천
3,KNHMB76137S128033,2019,3,수도권,서울
4,KNHUP7513YS048282,2019,6,수도권,서울
...,...,...,...,...,...
106467,KMHMJ81VP7U300058,2020,4,수도권,서울
106468,KMHWP81JP6U718394,2020,1,수도권,서울
106469,KN3HAP53E3K912032,2020,1,수도권외,부산
106470,KNAMA75433S415876,2020,5,수도권,인천


In [408]:
cdict = {
    '차대번호':'VIN', 
    '적발년도':'DSCL_YR', 
    '적발건수':'DSCL_NOCS', 
    '지역':'RGN',
    '시도':'CTPV', 
}

In [409]:
STD_BD_ORDITM_DSCL_CURSTT = orditm.rename(columns=cdict)
STD_BD_ORDITM_DSCL_CURSTT.columns

Index(['VIN', 'DSCL_YR', 'DSCL_NOCS', 'RGN', 'CTPV'], dtype='object')

#### [출력] STD_BD_ORDITM_DSCL_CURSTT

In [ ]:
# expdf = STD_BD_ORDITM_DSCL_CURSTT
# table_nm = 'STD_BD_ORDITM_DSCL_CURSTT'.upper()

# # 테이블 생성
# sql = 'create table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# wbt.execute(sql)

# # 데이터 추가
# # 5s
# wbt.import_from_pandas(expdf, table_nm)

### 적발지역별 계절제 단속 현황 출력

In [410]:
is_total.head()

,적발번호,발령번호,등록시도코드,등록시군구코드,적발지역코드,차대번호,단속일,농도기준아이디,유형기준아이디,등록시도,등록시군구,적발시도,적발시도
0,54793831,1838,41,190,41190,KMFXKS7BP3U657943,NaN,C012,T001,경기도,부천시,경기도,부천시
1,54793839,1838,41,190,41190,KNAMA75432S388470,NaN,C012,T001,경기도,부천시,경기도,부천시
2,54793843,1838,28,237,41190,KNGDNM9N14K178854,NaN,C012,T001,인천광역시,부평구,경기도,부천시
3,54795717,1838,41,130,41190,KMFGA17PP7C072098,NaN,C012,T001,경기도,성남시,경기도,부천시
4,54793837,1838,41,190,41190,KPBSH3AB18P082348,NaN,C012,T001,경기도,부천시,경기도,부천시


In [148]:
slimit.head()

,적발지역,적발시도,차대번호,계절제_1차,계절제_2차,계절제_3차,계절제_4차,차종,차종유형
0,수도권,서울,KMFZCS7JP7U292779,4.00,NaN,NaN,NaN,화물,소형
1,수도권,인천,KMHSD81VP2U244691,1.00,3.00,NaN,NaN,승용,중형
2,수도권,인천,KMHJN81VP6U261521,1.00,NaN,NaN,NaN,승용,중형
3,수도권,인천,KNAJC52135A471475,1.00,NaN,NaN,NaN,승용,대형
4,수도권,경기,KMHSJ81WP6U029832,1.00,4.00,NaN,NaN,승용,대형


In [149]:
slimit.columns

Index(['적발지역', '적발시도', '차대번호', '계절제_1차', '계절제_2차', '계절제_3차', '계절제_4차', '차종',
       '차종유형'],
      dtype='object')

In [150]:
cdict = {
    '적발지역':'DSCL_RGN', 
    '적발시도':'DSCL_CTPV', 
    '차대번호':'VIN', 
    '계절제_1차':'SEASON_1ODR_CRDN_NOCS', 
    '계절제_2차':'SEASON_2ODR_CRDN_NOCS', 
    '계절제_3차':'SEASON_3ODR_CRDN_NOCS', 
    '계절제_4차':'SEASON_4ODR_CRDN_NOCS', 
    '차종':'VHCTY_CD',
    '차종유형':'VHCTY_TY', 
}

In [151]:
STD_BD_SEASON_DSCL_RGN_CURSTT = slimit.rename(columns=cdict)
STD_BD_SEASON_DSCL_RGN_CURSTT.columns

Index(['DSCL_RGN', 'DSCL_CTPV', 'VIN', 'SEASON_1ODR_CRDN_NOCS',
       'SEASON_2ODR_CRDN_NOCS', 'SEASON_3ODR_CRDN_NOCS',
       'SEASON_4ODR_CRDN_NOCS', 'VHCTY_CD', 'VHCTY_TY'],
      dtype='object')

#### [출력] [D] STD_BD_SEASON_DSCL_RGN_CURSTT

In [ ]:
# expdf = STD_BD_SEASON_DSCL_RGN_CURSTT
# table_nm = 'STD_BD_SEASON_DSCL_RGN_CURSTT'.upper()

# # 테이블 생성
# sql = 'create table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# wbt.execute(sql)

# # 데이터 추가
# # 5s
# wbt.import_from_pandas(expdf, table_nm)

# code end